In [1]:
from dejavu import Dejavu

#print Dejavu.__version__
config = {"database":{
    "host" : "127.0.0.1",
    "user": "root",
    "passwd": "",
    "db":"dejavu",
    }
}
djv = Dejavu(config)


/anaconda2/lib/python2.7/site-packages/dejavu/database_sql.py:158: Warning: Table 'songs' already exists
  cur.execute(self.CREATE_SONGS_TABLE)
/anaconda2/lib/python2.7/site-packages/dejavu/database_sql.py:159: Warning: Table 'fingerprints' already exists
  cur.execute(self.CREATE_FINGERPRINTS_TABLE)


In [25]:
def SURFCount(image, name, th):
    img = image.copy()
    surf = cv2.xfeature2d.SURF_create(900)
    kp1, des1 = surf.detectAndCompute(img, None)
    for i in range(3):
        des2 = np.load('graph/'+name+'.png_cut'+str(i+1)+'.npy')
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3)
        search_params = dict(checks=5)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        good = []
        for i,(m,n) in enumerate(matches):
            if m.distance <= 0.75* n.distance:
                good.append(1)
        #print(time.time() - time0)
        if len(good)>th:
            return name
    for i in range(2):
        des2 = np.load('graph/'+name+'.png_window'+str(i+1)+'.npy')
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3)
        search_params = dict(checks=5)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        good = []
        for i,(m,n) in enumerate(matches):
            if m.distance <= 0.75* n.distance:
                good.append(1)
        #print(time.time() - time0)
        if len(good)>th:
            return name
    return 0

In [ ]:
import BaseHTTPServer
from SocketServer import BaseServer, ThreadingTCPServer
import threading
from SocketServer import ThreadingMixIn
import urllib
import os
import urlparse
import base64
from pydub import AudioSegment
import re
import librosa
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
from dejavu.recognize import FileRecognizer
import cv2
import time


class WebRequestHandler(BaseHTTPServer.BaseHTTPRequestHandler):

    def do_GET(self):
        thread = threading.currentThread()
        self.send_response(200)
        self.send_header("Content-type","text/plain")
        self.end_headers()


    def do_POST(self):
        thread = threading.currentThread()
        if self.rfile:
            time0 = time.time()
            length = 0
            for key,value in dict(urlparse.parse_qs(self.rfile.read(int(self.headers['Content-Length'])))).items():
                if key=='length':
                    print value
                    length = value[0]
                else:
                    
                    #print value[0]
                    #print len(value[0])
                    a = base64.decodestring(value[0])
        #        print key
                
                    out = open('test.wav', 'wb')
                    out.write(a)
                    out.close()
                    newAudio = AudioSegment.from_wav("test.wav")
                        
                    #newAudio3 = newAudio[2500:5000]
                    newAudio1 = newAudio[0:5000]
                    newAudio1.export('newRecord.wav', format="wav")
                    song = djv.recognize(FileRecognizer, 'newRecord.wav')
                    if song == None:
                        text = "No Match"
                    elif song['confidence']>=10:
                        text = song['song_name']
                    else:
                        newAudio[0:2500].export('Record_for_shazam1.wav', format="wav")
                        newAudio[2500:5000].export('Record_for_shazam2.wav', format="wav")
                        #newAudio4 = newAudio[3000:5000]
                        song1 = djv.recognize(FileRecognizer, 'Record_for_shazam1.wav')
                        song2 = djv.recognize(FileRecognizer, 'Record_for_shazam2.wav')
                        #song3 = djv.recognize(File)
                        if song1 is None and song2 is None:
                            text = 'No Match'
                        elif song1 is None and song2['confidence']>=3:
                            text = song2['song_name']
                        elif song2 is None and song1['confidence']>=3:
                            text = song1['song_name']
                        elif song1['confidence']<3 and song2['confidence']<3:
                            text = 'No Match'
                        #can modify the number of slice and the confidence 
                        elif song1['song_id'] == song2['song_id']:
                            text = song1['song_name']
                        else:    
                            newAudio1 = newAudio[0:5000]
                            newAudio1.export('newRecord.wav', format="wav")
                            y, sr = librosa.load('newRecord.wav')
                            librosa.feature.melspectrogram(y=y, sr=sr)
                            D = np.abs(librosa.stft(y))**2
                            S = librosa.feature.melspectrogram(S=D)
                            # Passing through arguments to the Mel filters
                            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
                            #print(int(len(y)/(sr)))
                            plt.figure(figsize=(int(len(y)/(sr)),40))
                            librosa.display.specshow(librosa.power_to_db(S,ref=np.max), fmax=8000)
                            #plt.colorbar(format='%+2.0f dB')
                            #plt.title('Mel spectrogram')
                            plt.tight_layout()
                            plt.savefig("test.png", bbox_inches='tight', pad_inches = 0)
                            image = cv2.imread('test.png')
                            #th can be modified
, and I belive using ratio will be better, will explore on that                            name1 = SURFCount(image, song1['song_name'],15)
                            if name1!=0:
                                text = name1
                            else:
                                name2 = SURFCount(image, song2['song_name'],15)
                                if name2 !=0:
                                    text = name2
                                else:
                                    text = 'No Match'
                                               
                # print content
        self.send_response(200)
        self.send_header("Content-type","text/plain")
        self.end_headers()
        print (time.time()-time0)
        self.wfile.write(text)
            

class ThreadingServer(ThreadingMixIn, ThreadingTCPServer):
    pass

if __name__ == '__main__':
    server = ThreadingServer(('192.168.1.4', 4636), WebRequestHandler)
    thread = threading.Thread(target=server.serve_forever)
    thread.start()
    print "Server Starts"
    while True:
        continue
